## Workflow (do not run)

Original data source - https://ood.rc.ufl.edu/pun/sys/dashboard/files/fs//blue/stevenweisberg/share/ai_catalyst_silcton/derivatives/sourcedata/freesurfer

copied to - https://ood.rc.ufl.edu/pun/sys/dashboard/files/fs//blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/freesurfer

Mask created using aparc+aseg.mgz -> aparc+aseg_standardized.mgz, same from brain data, brain.mgz -> brain_standardized.mgz

For participants with default 1mm3 size, standardized files are just copies

In [2]:
# Updated Python script for proper 1mm³ resampling
# MAINLY TO DOWNSAMPLE APARC+ASEG.MGZ FOR 0.8MM3 TO 1MM3 TO BE IN LINE WITH OTHER PARTICIPANTS

# Define the list of subject IDs
subjects = [
    1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062,
    1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072,
    1074, 1075, 1076, 1077, 1078, 1079, 1080
]

# Define a function to generate commands (2 commands per subject)
def generate_commands(subject_id):
    base_path = f"/blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/freesurfer/sub-{subject_id}/mri"
    
    # First command: create 1mm³ template
    cmd1 = f"mri_convert {base_path}/orig.mgz {base_path}/orig_1mm.mgz -c"
    
    # Second command: use mri_label2vol with the 1mm³ template
    cmd2 = (f"mri_label2vol --seg {base_path}/aparc+aseg.mgz --temp {base_path}/orig_1mm.mgz "
            f"--regheader {base_path}/aparc+aseg.mgz --o {base_path}/aparc+aseg_standardized.mgz --fillthresh 0.3")
    
    return [cmd1, cmd2]

# Generate all commands
all_commands = []
for subject in subjects:
    all_commands.extend(generate_commands(subject))

# Define the output file name
output_file = "aparc+aseg_standardized_fixed.txt"

# Save the commands to the output file
with open(output_file, "w") as file:
    for command in all_commands:
        file.write(command + "\n")

print(f"Commands saved to {output_file}")
print(f"Total commands generated: {len(all_commands)}")

In [ ]:
# MAINLY TO DOWNSAMPLE BRAIN.MGZ FOR 0.8MM3 TO 1MM3 TO BE IN LINE WITH OTHER PARTICIPANTS

# Define the list of subject IDs
subjects = [
    1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062,
    1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072,
    1074, 1075, 1076, 1077, 1078, 1079, 1080
]

# Define a function to generate the mri_convert command
def generate_mri_convert_command(subject_id):
    input_path = f"/blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/freesurfer/sub-{subject_id}/mri/brain.mgz"
    output_path = f"/blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/freesurfer/sub-{subject_id}/mri/brain_standardized.mgz"
    return f"mri_convert -i {input_path} -o {output_path} -c"

# Generate the mri_convert commands for all subjects
mri_convert_commands = [generate_mri_convert_command(subject) for subject in subjects]

# Define the output file name
output_file = "brain_standarized.txt"

# Save the commands to the output file
with open(output_file, "w") as file:
    for command in mri_convert_commands:
        file.write(command + "\n")

print(f"Commands saved to {output_file}")


Binarize the mask - hippocampusmask_list = ["mri_binarize --i /blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/freesurfer/{}/mri/aparc+aseg_standardized.mgz --match 17 53 --o /blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/freesurfer/{}/mri/hippocampus_new_standardized_mask.nii.gz".format(subj, subj) for subj in subjs]


apply mask

In [3]:
import nibabel as nib
import numpy as np

# Lists containing filenames
#raw_list = ['rawavg1.mgz', 'rawavg2.mgz', 'rawavg3.mgz']  # Replace with your rawavg_img filenames
#mask_list = ['mask1.nii.gz', 'mask2.nii.gz', 'mask3.nii.gz']  # Replace with your hippocampus_mask filenames
#output_list = ['masked_output1.nii.gz', 'masked_output2.nii.gz', 'masked_output3.nii.gz']  # Replace with desired output filenames

for raw_file, mask_file, output_file in zip(brain_list, mask_list, output_list):
    # Load the MRI image and mask
    rawavg_img = nib.load(raw_file)
    mask_img = nib.load(mask_file)

    # Get the data arrays from the images
    rawavg_data = rawavg_img.get_fdata()
    mask_data = mask_img.get_fdata()

    # Apply the mask
    masked_data = np.where(mask_data == 1, rawavg_data, 0)

    # Create a new NIfTI image with the masked data
    masked_img = nib.Nifti1Image(masked_data, rawavg_img.affine)

    # Save the masked image
    nib.save(masked_img, output_file)

    print(f"Masking completed for {raw_file}. Output saved as {output_file}")


then normalize

In [ ]:
import os
from nibabel import save, load
import pandas as pd
import nibabel
from pandas import read_csv
import numpy as np


data_dir = '/blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/amygdala_standardized'
save_dir = '/blue/stevenweisberg/ashishkumarsahoo/difumo/Data/freesurfer_121023/amygdala_standardized_normalized'

subjects = read_csv(r'/blue/stevenweisberg/ashishkumarsahoo/hippocampAI/participants.tsv',sep='\t',header=0)
subjs = subjects.participant_id

        
for subj in subjs:
    subj_number = subj[4:]
    t1_image_MNI = load('{}/sub-{}_amygdalamasked_brain_standardized.nii.gz'.format(data_dir,subj_number))
    isExist = os.path.exists('{}/{}'.format(data_dir,subj))
    if not isExist:
        os.makedirs('{}/{}'.format(data_dir,subj))
    img_data = t1_image_MNI.get_fdata()
    points = np.argwhere(img_data)
    # Normalize intensity with respect to the maximum value
    #max_value = np.max(img_data)
    min_value = np.min(img_data)
    print ('min value for', subj_number, ' :', min_value)
    max_value = np.max(img_data)
    print ('max value for', subj_number, ' :', max_value)
    #if min_value >= 0:
        #print('error')
    #print ('max value for', subj_number, ' :', max_value)
    #img_data = img_data-min_value #REMOVED SINCE NO NEGATIVE VALUES EXIST
    max_value = np.max(img_data)
    normalized_data = img_data / max_value
    if np.any(normalized_data < 0):
        print('error')
    np.save('{}/sub-{}_amygdalamasked_brain_standardized_noneg_normalized.npy'.format(save_dir,subj_number), normalized_data)
    #print(subj)